# Tutorial Notebook

## 0. Setup and load data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fasttreeshap
import matplotlib.pyplot as plt
import pandas as pd
import shap

from src.utils import _eval_pred, _fit_pipeline, _get_X_y_idx, _set_train_test

"is" with a literal. Did you mean "=="?
"is" with a literal. Did you mean "=="?
"is not" with a literal. Did you mean "!="?
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


[Errno 2] No such file or directory: '/Users/sfchen/Documents/2018-2024_Scripps/Torkamani_Lab/230404_draft/CAD_meta_prediction/.venv/lib/python3.10/site-packages/lohrasb/config.yaml'
In this module, the default logging will be applied. The error is [Errno 2] No such file or directory: '/Users/sfchen/Documents/2018-2024_Scripps/Torkamani_Lab/230404_draft/CAD_meta_prediction/.venv/lib/python3.10/site-packages/lohrasb/config.yaml' which will be skipped!
default logger setting is applied !


In [3]:
y_label = "cardio"
id_col = "id"
trainer = ("xgb", "classifier")
n_features = 5
n_trials = 100

In [4]:
df = pd.read_csv("data/cardio_train.csv", sep=";")
categorical_cols = ["gender", "cholesterol", "gluc", "smoke", "alco", "active", "cardio"]
df[categorical_cols] = df[categorical_cols].astype("object")
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


## 1. ML pipeline execution

In [5]:
X, y, idx = _get_X_y_idx(df=df, id_col=id_col, y_label=y_label, trainer=trainer)

input data shape: (70000, 12)
input outcome shape: (70000, 1)


In [6]:
X_train, X_test, y_train, y_test, _, idx_test = _set_train_test(trainer, X, y, idx)

In [ ]:
fitted_pipeline = _fit_pipeline(
    X_train=X_train,
    y_train=y_train,
    trainer=trainer,
    fname=f"{trainer[0]}_{trainer[1]}__{y_label}",
    n_features=n_features,
    n_trials=n_trials,
)

measure_of_accuracy = f1_score(y_true, y_pred, average='macro')

pre-SHAP init:
- X_train.shape:(56000, 12)
- y_train.shape(56000, 1)


[I 2025-04-14 02:26:39,462] A new study created in memory with name: feature_selection
[I 2025-04-14 02:26:43,735] Trial 0 finished with value: 0.39237277234486095 and parameters: {'objective': 'binary:logistic', 'eval_metric': 'auc', 'gamma': 3, 'learning_rate': 0.13103143857073918, 'max_bin': 18, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 101, 'n_jobs': -1, 'random_state': 19920722, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 17.485604649374103}. Best is trial 0 with value: 0.39237277234486095.


In [ ]:
y_pred, acc = _eval_pred(pip=fitted_pipeline, trainer=trainer, X=X_test, y=y_test)

## 2. SHAP explanation of best estimator

In [ ]:
plt.cla()
plt.clf()
fig = plt.gcf()

scaled_X_test = fitted_pipeline[:-1].transform(X_test)
estimator = fitted_pipeline[-1].best_estimator
explainer = fasttreeshap.TreeExplainer(estimator, algorithm="v2", n_jobs=-1)
shap_values = explainer.shap_values(scaled_X_test)

shap.summary_plot(shap_values, scaled_X_test, max_display=n_features, show=True)
plt.close()
plt.cla()
plt.clf()